This notebook will walk you through the important tables in the qc schema and how to use them to filter the sessions you need

In [ ]:
# import datajoint and modules from ibl_pipeline
import datajoint as dj
from ibl_pipeline import reference, subject, acquisition, behavior
from ibl_pipeline.analyses import behavior as behavior_analyses
import numpy as np

QC tables are still in active development. We therefore recommend accessing them with `dj.create_virtual_module()`

In [ ]:
qc = dj.create_virtual_module('qc', 'ibl_qc')

# QC tables

In [ ]:
dj.Diagram(qc) + acquisition.Session

This diagram shows the QC related tables, and here is the description of the tables.


>* QCChoice: Available flags to quantify the quality of a session or a specific aspect of a session, lookup table got referred in SessionQC and SessionExtendedQC, 50 for CRITICAL, 40 for FAIL, 30 for WARNING, 10 for PASS, and 0 for NOT SET  
>* SessionQC: QCChoice for each session, ingested from alyx field `qc` in the table `actions.session`  
>* QCType:  Aspect of a session for quality check. e.g. task, behavior, experimenter…  
>* SessionExtendedQC: QCChoice (e.g. FAIL) for a QCType (e.g. task) for each session, structured data about SessionQC  
>* SessionExtendedQC.Field: Part table of SessionExtendedQC. For each entry of SessionExtendedQC, there may be multiple fields describing each value (e.g. 0.99) of a qc aspect (e.g. _task_stimOn_delays) that belongs to a QCType (e.g. task).



Detailed table definitions could be easily checked with the method `describe()`, for example

In [ ]:
qc.QCChoice.describe();

Preview the contents of the table:

In [ ]:
qc.QCChoice()

# Useful queries

In [ ]:
# Sessions better than CRITICAL?
acquisition.Session & (qc.SessionQC & 'qc < 50')

In [ ]:
# Sessions better than critical and also good enough for brainwide map?

acquisition.Session & (qc.SessionQC & 'qc < 50') & \
(behavior_analyses.SessionTrainingStatus & 'good_enough_for_brainwide_map')

In [ ]:
# Sessions better than critical for task criteria?

acquisition.Session & (qc.SessionExtendedQC & 'qc_type="task"' & 'extended_qc < 40')


# Summary

In this notebook, we listed a few query examples related to the qc schema that might be helpful for your research. For a full fledged introduction of major types of queries and fetches, please refer to [this notebook](../201909_code_camp/1-Explore%20IBL%20data%20pipeline%20with%20DataJoint.ipynb) during the 2019 IBL Code Camp.